In [122]:
import tensorflow as tf
import numpy as np
from music21 import *
import os
import json
import numpy as np
import pickle
from random import shuffle
import zipfile
import gensim
import random
from datetime import datetime
import matplotlib.pyplot as plt
import shutil

In [114]:
def build_discriminator(music,num_neurons, num_dense_neurons):
    bidir = tf.keras.layers.Bidirectional(tf.keras.layers.CuDNNLSTM(num_neurons), input_shape=(3072, 50))
    hidden_dense = tf.keras.layers.Dense(num_dense_neurons, activation = tf.nn.relu)
    condense = tf.keras.layers.Dense(1)
    sigmoid = tf.keras.layers.Activation('sigmoid')
    
    layer1 = bidir(music)
    layer2 = hidden_dense(layer1)
    layer3 = condense(layer2)
    final_output = sigmoid(layer3)
    return tf.keras.Model(music, final_output)

In [115]:
def build_generator(original_input_music, input_music):
        
        hidden_output = tf.keras.layers.CuDNNLSTM(200, activity_regularizer = tf.math.sigmoid)(input_music)
        
        repeated_output = tf.keras.layers.RepeatVector(1536)(hidden_output)
        
        #end_sequence = tf.compat.v1.keras.layers.CuDNNLSTM(50, return_sequences = True, activity_regularizer = tf.math.sigmoid)(repeated_output)
        
        end_sequence = tf.keras.layers.LSTM(50, return_sequences = True, activation = tf.math.sigmoid)(repeated_output)
        results = tf.keras.layers.Concatenate(1)([input_music, end_sequence])
        return tf.keras.Model(original_input_music, results), results
    

In [116]:
def next_batch(lis, length):
    counter = 0
    string = ""
    for item in lis:
        string = str(item)
        zipfilePath = ("./clean_data/" + string + ".zip")
        zip = zipfile.ZipFile(zipfilePath)
        zip.extractall("./clean_data")
        pickle_file = open("./clean_data/clean_data/" + string + ".pickle","rb")
        lis = pickle.load(pickle_file)
        pickle_file.close()
        ray = np.array(lis)
        
        ray = np.transpose(ray)
        index = random.randint(0,ray.shape[0]-1-length)
        ray = ray[index:index+length, :]
        if counter == 0:
            final = np.reshape(ray, [1,length,88])
        else:
            final = np.concatenate((final, np.reshape(ray, [1,length,88])), 0)
        shutil.rmtree("./clean_data/clean_data")
        counter = counter + 1
    return (final)

In [ ]:
plt.ion()
plt.figure(figsize = (15,10))
fig = plt.gcf()
fig.show()
fig.canvas.draw()


In [117]:
with tf.device('/gpu:0'):
    music = tf.keras.Input([3072,88])
    input_music = tf.keras.Input([1536,88])

    coded_input_music = tf.keras.Input([3072,50])

    test_discrim_input = tf.keras.Input([3072,50])

    encode = tf.keras.layers.Dense(50, activation = "relu")
    decode = tf.keras.layers.Dense(88, activation = 'sigmoid')

    encoded_dis_music = tf.keras.layers.TimeDistributed(encode)(music)
    encoded_gen_music_input = tf.keras.layers.TimeDistributed(encode)(input_music)

    decoded_music = tf.keras.layers.TimeDistributed(decode)(encoded_dis_music)
    decoded_input_music_for_model = tf.keras.layers.TimeDistributed(decode)(coded_input_music)

    decoder_model = tf.keras.Model(coded_input_music, decoded_input_music_for_model)

    decoded_input_music = tf.keras.layers.TimeDistributed(decode)(encoded_gen_music_input)

    encoder_decoder = tf.keras.Model(inputs = [music, input_music], outputs = [decoded_music, decoded_input_music])

    encoder_decoder.compile(optimizer = 'adam' , loss='categorical_crossentropy', loss_weights = [.5,.5])

    encode.trainable = False
    decode.trainable = False

In [118]:
with tf.device('/gpu:0'):
    discriminator = build_discriminator(test_discrim_input, 128, 50)

    real_discriminator_output = discriminator(encoded_dis_music)

    real_discriminator = tf.keras.Model(music, real_discriminator_output)

    discriminator.compile(optimizer = 'adam', loss='binary_crossentropy')
    real_discriminator.compile(optimizer = 'adam', loss='binary_crossentropy')

In [119]:
with tf.device('/gpu:0'):
    generator, to_dis = build_generator(input_music, encoded_gen_music_input)
    discriminator.trainable = False
    real_discriminator.trainable = False
    fake_lables = discriminator(to_dis)
    combined = tf.keras.Model(input_music, fake_lables)
    combined.compile(optimizer = 'adam', loss='binary_crossentropy')

In [120]:
num_total_data = 15509
batch_size = 500
epochs = 200

In [1]:
loss_dict= {
    'e_loss':[],
    'g_loss':[],
    'd_loss':[]
    
}
for e in range(epochs):
    num_batches = num_total_data // batch_size
    dis_list = list(range(num_total_data))
    gen_list = list(range(num_total_data))
    random.shuffle(dis_list)
    random.shuffle(gen_list)
    for i in range(num_batches):
        print("getting data")
        start_data_gen = next_batch(gen_list[i*batch_size:i*batch_size + batch_size], 1536)
        sequences_dis = next_batch(dis_list[i*batch_size:i*batch_size + batch_size], 3072)
        print("got data")
        encoder_loss = encoder_decoder.train_on_batch([sequences_dis,start_data_gen], [sequences_dis, start_data_gen])
        total_fake_data = generator.predict(start_data_gen)
        d_loss_fake = discriminator.train_on_batch(total_fake_data, tf.zeros(batch_size))
        d_loss_real = real_discriminator.train_on_batch(sequences_dis, tf.ones(batch_size)*.9)
        print(real_discriminator.predict(sequences_dis))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        g_loss = combined.train_on_batch(start_data_gen, tf.ones(batch_size))
        g_test = decoder_model.predict(generator.predict(start_data_gen))
        print(g_test)
        print("batch {} completed! encoder loss: {} discriminator loss: {} generator_loss: {}".format(i, encoder_loss[0],d_loss,g_loss))
        loss_dict['e_loss'].append(encoder_loss[0])
        loss_dict['g_loss'].append(g_loss)
        loss_dict['d_loss'].append(d_loss)
        plt.clf()
        plt.subplot(3,1,1)
        plt.plot(loss_dict['e_loss']) # plot something
        plt.ylabel("e_loss")
        plt.subplot(3,1,2)
        plt.plot(loss_dict['g_loss'])
        plt.ylabel("g_loss")
        plt.subplot(3,1,3)
        plt.plot(loss_dict['d_loss'])
        plt.ylabel("d_loss")
        plt.pause(0.01)
        fig.canvas.draw()
        if i % 10 == 0:
            print("10 batches completed")
            single_start_data = next_batch([random.randint(0,num_total_data)], 1536)
            sample_new_data = decoder_model.predict(generator.predict(single_start_data))
            with open("./music_samples/{}.pickle".format(i), "wb+") as file:
                pickle.dump(sample_new_data, file)
        encoder_decoder.save("./models/encoder_decoder{}&{}.h5".format(e,i))
        discriminator.save("./models/discriminator{}&{}.h5".format(e,i))
        real_discriminator.save("./models/discriminator{}&{}.h5".format(e,i))
        generator.save("./models/generator{}&{}.h5".format(e,i))
        combined.save("./models/combined{}&{}.h5".format(e,i))
                              
                        

NameError: name 'epochs' is not defined